# Economic Planning

## Objective and Prerequisites

In this example, you’ll discover how mathematical optimization can be used to address a  macroeconomic planning problem that a country may face. We’ll show you how to model and solve an input-output problem encompassing the interrelationships between the different sectors of a country’s economy.

This model is example 9 from the fifth edition of Model Building in Mathematical Programming by H. Paul Williams on pages 263-264 and 316-317.

This modeling example is at the intermediate level, where we assume that you know Python and are familiar with the Gurobi Python API. In addition, you should have some knowledge about building mathematical optimization models.

**Download the Repository** <br /> 
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). 

## Problem Description

In this problem, we assume that we have an economy with three types of industries:

* Coal
* Steel
* Transport

Part of the outputs from these industries are needed as inputs to others. For example, coal is needed to fire the blast furnaces that produce steel, steel is needed in the machinery for extracting coal, etc. We measure all units of production in monetary terms.  That is, each dollar of production by one of the industries requires inputs (in dollars) from possibly its own industry as well as other industries. The required inputs as well as the labor requirements (also measured in dollars) are shown in the following table. 

| input (t) / <br /> output (t+1) | Coal | Steel | Transport |
| --- | --- | --- | --- | 
| Coal | 0.1 | 0.5 | 0.4 | 
| Steel | 0.1 | 0.1 | 0.2 |
| Transport | 0.2 | 0.1 | 0.2 |
| Labor | 0.6 | 0.3 | 0.2 | 
|**Total**  | **1** | **1** | **1**|

There is a time lag in the economy so that the output in year t +1 requires an input in year t.  For example, to produce one-dollar worth of coal requires 0.1 dollars of coal (to provide the necessary power), 0.1 dollars of steel (the steel ‘used up’ in the ‘wear and tear’ on the machinery) and 0.2 dollars of transport (for moving the coal from the mine). In addition, 0.6 dollars of manpower is required. Similarly, the other columns of the table give the inputs required (in dollars) for each dollar of steel and each dollar of transport (trucks, cars, trains, etc.). Notice that the value of each unit of output is exactly matched by the sum of the values of its inputs.

Output from an industry may also be used to build productive capacity for itself or other industries in future years. The inputs required to give unit increases (capacity in dollars’ worth of extra production) in productive capacity are given in the following table. Input from an industry in year t results in a (permanent) increase in productive capacity in year t +2.

| input (t) / <br /> output (t+2) | Coal | Steel | Transport |
| --- | --- | --- | --- | 
| Coal | 0.1 | 0.7 | 0.9 | 
| Steel | 0.1 | 0.1 | 0.2 |
| Transport | 0.2 | 0.1 | 0.2 |
| Labor | 0.4 | 0.2 | 0.1 | 

Stocks of goods may be held from year to year. At present (the beginning of year 1), the stocks and productive capacities (per year) are given in the following table (in millions of dollars).

| Present | Stocks | Productive Capacity |
| --- | --- | --- | 
| Coal | 150 | 300 | 
| Steel | 80 | 350 | 
| Transport | 100 | 280 |  

We want to maximize the total manpower utilization i.e. employment, over the planning horizon while meeting an exogenous consumption requirement in every year:

* 60 million dollars of coal. 
* 60 million dollars of steel.
* 30 million dollars  of transport.


## Model Formulation
A widely used type of national economic model is the input–output model representing the interrelationships between the different sectors of a country’s economy. Such models are often referred to as dynamic Leontief models after their originator, who built such a model of the American economy. A similar model has been considered by Wagner (1957).

### Sets and indices

$i,j \in \text{Industries}=\{\text{coal}, \text{steel}, \text{transport}\}$

$ t \in \text{Horizon} = \{\text{year1}, \text{year2}, \text{year3}, \text{year4}, \text{year5}, \text{year6} \}$ 

$ t \in \text{fiveYears} = \{\text{year1}, \text{year2}, \text{year3}, \text{year4}, \text{year5} \}$  


$t \in H_{2,4} = \{year2, year3, year4 \} $

### Parameters

$\text{demand}_{j} \in \mathbb{R}^+$: Exogenous demand of goods in industry $j$.

$\text{initial_stock}_{j} \in \mathbb{R}^+$: Stocks of goods in industry $j$ available at the beginning of year 1.

$\text{in_out_prod}_{i,j} \in \mathbb{R}^+$: Input of good $i$ required to produce one unit of good $j$ in the following year.

$\text{in_out_cap}_{i,j} \in \mathbb{R}^+$: Input of good $i$ in current year results in a permanent increase in productive capacity of good $j$  two years later.

$\text{industry_cap}_{j} \in \mathbb{R}^+$: Industry $j$  productive capacity at the beginning of year 1.

$\text{labor_prod}_{j} \in \mathbb{R}^+$: Labor requirements for the production of goods in industry  $j$.

$\text{labor_extra_cap}_{j} \in \mathbb{R}^+$: Labor requirements to permanently increase capacity for goods in industry $j$.

### Decision Variables

$\text{production}_{j,t} \in \mathbb{R}^+$: Production of goods in industry $j$ available at year $t$, in millions of dollars.
Note: goods of industry j, in millions of dollars, available in year t but produced in previous year.

$\text{stock}_{j,t} \in \mathbb{R}^+$: Stock level of industry $j$ at the end of year $t$, in millions of dollars.

$\text{extra_cap}_{j,t} \in \mathbb{R}^+$: Extra capacity for industry   $j$ becoming effective at the beginning of year $t$, in millions of dollars.


In order to build a realistic model, we need to think beyond year 5. Therefore, we make the following assumptions:

* External demand is constant up to year 5 and beyond.
* Stock level remains constant at year 5 and beyond.
* Capacity cannot be increased after year 5.

Consequently, for year 6 and beyond, we can assume a static model of the economy. For this static model, the production of goods in each industry $i \in \text{Industries}$ can be computed as follows.

\begin{equation}
x_{i} = \text{demand}_{i} + \sum_{j \in \text{Industries} } \text{in_out_prod}_{i,j} * x_{j}
\end{equation}

Where $x_{i}$ is the production of goods in industry $i$.

Solving this system of equations gives lower bounds for the production of goods in years beyond year 5. Then, the production of goods of  industry $j \in \text{Industries}$ in year 6 and beyond can be defined by the following constraints. 

$$
\text{production}_{j,t} \geq x_{j} \quad \forall j \in \text{Industries}, \; t=6. 
$$

$$
\text{extra_cap}_{j,t} = 0 \quad \forall j \in \text{Industries}, \; t=6.
$$


### Constraints

**Balance equation year 1**: The initial stocks in industry $i$ should be equal to the total demand (internal and external), plus the extra capacity to build, plus the stock level at the end of year 1 of the goods in that industry.

\begin{equation}
\text{initial_stock}_{i} = \sum_{j \in \text{Industries} } \text{in_out_prod}_{i,j}*\text{production}_{j,2}
+ \text{demand}_{i} + \sum_{j \in \text{Industries} } \text{in_out_cap}_{i,j}*\text{extra_cap}_{j,3} +
 \text{stock}_{i,1}
\end{equation}

**Balance equation for year 2, year 3, year 4**: The production of goods in industry $i$ at year t plus the stocks at the end of year t-1 should be equal to the total demand (internal and external), plus the extra capacity to build, plus the stock level at the end of year t of the goods in that industry. 


\begin{equation}
\text{production}_{j,t} + \text{stock}_{i,t-1} =
\sum_{j \in \text{Industries} } \text{in_out_prod}_{i,j}*\text{production}_{j,t+1} + \text{demand}_{i} +
\sum_{j \in \text{Industries} } \text{in_out_cap}_{i,j}*\text{extra_cap}_{j,t+2} 
+ \text{stock}_{i,t} \quad \forall t \in H_{2,4}
\end{equation}


**Balance equation year 5**: The production of goods in industry $i$ plus the stocks at the end of year 4 should be equal to the total demand (internal and external), plus the stock level at the end of year 5 of the goods in that industry.

\begin{equation}
\text{production}_{j,5} + \text{stock}_{i,4} =
\sum_{j \in \text{Industries} } \text{in_out_prod}_{i,j}*\text{production}_{j,6}  +
\text{demand}_{i} + \text{stock}_{i,5}
\end{equation}


**End of horizon constraints**: 

$$
\text{production}_{j,t} \geq x_{j} \quad \forall j \in \text{Industries}, \; t=6 
$$

$$
\text{extra_cap}_{j,t} = 0 \quad \forall j \in \text{Industries}, \; t=6
$$

Where $x_{j}$ is the solution of the static model.


**Productive capacity constraints**: These constraints ensure that the production of goods for each industry $j$ during the planning horizon do not exceed the total production capacity at that year. 

\begin{equation}
\text{production}_{j,t} \leq \text{base_cap}_{j} + \sum_{\tau \leq t} \text{extra_cap}_{j,\tau} \quad \forall t \in \text{Horizon}
\end{equation}

### Objective Function

**Labor utilization**: Maximize labor employment.

\begin{equation}
\text{Maximize} \quad Z =
\sum_{t \in \text{fiveYears} } \sum_{j \in \text{Industries} } \text{labor_prod}_{j}*\text{production}_{j,t} +
\sum_{t \in \text{fiveYears} } \sum_{j \in \text{Industries} } \text{labor_extra_cap}_{j}*\text{extra_cap}_{j,t}
\end{equation}


---
## Python Implementation

We import the Gurobi Python Module and other Python libraries.

In [ ]:
%pip install gurobipy

In [1]:
import pandas as pd
from itertools import product

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.1.0

## Input Data
We define all the input data for the model.

In [2]:
# input-output matrix for the production of goods of each industry

arcs, inout_prod = gp.multidict({
    ('coal', 'coal'): 0.1,
    ('coal', 'steel'): 0.5,
    ('coal', 'transport'): 0.4,
    ('steel', 'coal'): 0.1,
    ('steel', 'steel'): 0.1,
    ('steel', 'transport'): 0.2,
    ('transport', 'coal'): 0.2,
    ('transport', 'steel'): 0.1,
    ('transport', 'transport'): 0.2
})

# Labor requirements for the production of goods of each industry
labor_prod = dict({'coal': 0.6,
                   'steel': 0.3,
                   'transport': 0.2})

# input-output matrix to create extra capacity for each industry

arcs, inout_cap = gp.multidict({
    ('coal', 'coal'): 0.1,
    ('coal', 'steel'): 0.7,
    ('coal', 'transport'): 0.9,
    ('steel', 'coal'): 0.1,
    ('steel', 'steel'): 0.1,
    ('steel', 'transport'): 0.2,
    ('transport', 'coal'): 0.2,
    ('transport', 'steel'): 0.1,
    ('transport', 'transport'): 0.2
})

# Labor requirements to increase capacity for each industry
labor_extra_cap = dict({'coal': 0.4,
                  'steel': 0.2,
                  'transport': 0.1})

# Initial stock, initial capacity, and demand of goods for each industry

industries, stock0, capacity0, demand = gp.multidict({
    'coal': [250,300,60],
    'steel': [180,350,60],
    'transport': [200,280,30]
})

# Time Horizons
horizon = [1,2,3,4,5,6]
fiveYears = [1,2,3,4,5]
years2_4 = [2,3,4]

# Computed parameters
i2h = set(product(industries, horizon))
i2f = set(product(industries, fiveYears))

## Preprocessing

We assume a static model of the economy, for year 6 and beyond. This static model is defined by a system of equations. The decision variables are the production of goods of each industry for the static model of the economy. We formulate and solve this model. 

In [3]:
static = gp.Model('StaticModel')

static_prod = static.addVars(industries, name="static_prod")

# Static model balance equations

static_balance = static.addConstrs(
    (static_prod[i] - gp.quicksum(inout_prod[i,j]*static_prod[j] for j in industries) 
     == demand[i] for i in industries  ), name='static_balance' )

# we define a constant objective function to solve the system of balance equations for the static model.

static.setObjective(0)

# Verify model formulation

static.write("StaticModel.lp")

# Run optimization engine

static.optimize()

# Print solution of static model

print("\n\n_________________________________________________________________________________")
print(f"The production of goods by industry for the static model of the economy is:")
print("_________________________________________________________________________________")
for i in industries:
    if (static_prod[i].x > 1e-6):
        dollars_static_prod = '${:,.2f}'.format(static_prod[i].x)
        print(f"Generate {dollars_static_prod} million dollars of {i} ")

Using license file c:\gurobi\gurobi.lic
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3 rows, 3 columns and 9 nonzeros
Model fingerprint: 0x5b989667
Coefficient statistics:
  Matrix range     [1e-01, 9e-01]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 6e+01]
Presolve removed 3 rows and 3 columns
Presolve time: 0.05s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.05 seconds
Optimal objective  0.000000000e+00


_________________________________________________________________________________
The production of goods by industry for the static model of the economy is:
_________________________________________________________________________________
Generate $166.40 million dollars of coal

## Model Deployment

We create a model and the variables. For each year in the planning horizon and industry, the variables capture the production of goods, the stock level of goods, and the increase in production capacity. 

In [4]:
model = gp.Model('EconomicPlanning')

# Decision variables
production = model.addVars(i2h, ub=capacity0, name="production")

# There is no production at year 1
model.setAttr('ub', production.select('*',1), 0)    

stock = model.addVars(i2f, name="stock")
extra_cap = model.addVars(i2h, name="extra_cap")


# No extra capacity can be generated at years 1 and 2
for j,t in i2h:
    if t < 3:
        extra_cap[j,t].ub = 0  

The first constraint is a balance equation for year 1 of the planning horizon. The initial stocks in industry  $i$  should be equal to the total demand (internal and external), plus the extra capacity to build, plus the stock level at the end of year 1 of the goods in that industry.

In [5]:
# Year 1 balance equations 

balance1 = model.addConstrs( ( stock0[i] == gp.quicksum(inout_prod[i,j]*production[j,2] for j in industries)  
                              + gp.quicksum(inout_cap[i,j]*extra_cap[j,3] for j in industries ) 
                              + demand[i] + stock[i,1] for i in industries ), name='balance1' )

The following constraints are balance equations for years 2,3, and 4. The production of goods in industry  i  at year t plus the stocks at the end of year t-1 should be equal to the total demand (internal and external), plus the extra capacity to build, plus the stock level at the end of year t of the goods in that industry.

In [6]:
# Balance equations for years 2,3, and 4

balance_t = model.addConstrs(( production[i, year] + stock[i,year-1]  == 
                              gp.quicksum(inout_prod[i,j]*production[j, year + 1] for j in industries) 
                              + gp.quicksum(inout_cap[i,j]*extra_cap[j, year + 2] for j in industries ) 
                              + demand[i] + stock[i, year] for i in industries for year in years2_4 ), name='balance_t' )

The constraint for year 5 ensures that the production of goods in industry  i  plus the stocks at the end of year 4 should be equal to the total demand (internal and external), plus the stock level at the end of year 5 of the goods in that industry.

In [7]:
# Balance equations for year 5

balance5 = model.addConstrs( (production[i, 5] + stock[i,4]  == 
                              gp.quicksum(inout_prod[i,j]*production[j,6] for j in industries) 
                              + demand[i] + stock[i,5] for i in industries ), name='balance5' )

We need to enforce the constraints of a static model of the economy for year 6 and beyond.
* Production of goods at each industry should be larger or equal to the industry production of the static model of the economy.

* There is no increased capacity for year 6 and beyond.

In [8]:
# Steady state production for year 6 and beyond

steadyProduction = model.addConstrs((production[j,6] - static_prod[j].x >= 0 for j in industries ), name='steadyProduction')

# Zero increased capacity for year 6 and beyond
for j,t in i2h:
    if t == 6:
        extra_cap[j,t].ub = 0   

The productive capacity constraints ensure that the production of goods for each industry during the planning horizon do not exceed the total production capacity at that year.

In [9]:
# Productive capacity constraints

capacityConstr = model.addConstrs(
    (production[industry, year] - gp.quicksum(extra_cap[industry,t] for t in fiveYears if t <= year) 
     <= capacity0[industry] for industry,year in i2f ), name='capacityConstr' )

The objective is to maximize labor employment.

In [10]:
# Maximize employment
model.setObjective(
    (gp.quicksum(labor_prod[j]*production[j,t] for j in industries for t in fiveYears) 
     + gp.quicksum(labor_extra_cap[j]*extra_cap[j,t] for j in industries for t in fiveYears) ), GRB.MAXIMIZE)

In [11]:
# Verify model formulation

model.write('DynamicModel.lp')

# Run optimization engine

model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 33 rows, 51 columns and 183 nonzeros
Model fingerprint: 0x194e5768
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [1e-01, 6e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 4e+02]
Presolve removed 9 rows and 18 columns
Presolve time: 0.01s
Presolved: 24 rows, 33 columns, 126 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5258351e+04   7.671320e+03   0.000000e+00      0s
      25    1.9022169e+03   0.000000e+00   0.000000e+00      0s

Solved in 25 iterations and 0.01 seconds
Optimal objective  1.902216910e+03


---
## Analysis

The objective of maximizing employment results in a total manpower utilization of $\$1,902.22 $ millions. With this objective, the coal industry is boosted in view of its heavy labor requirements.

In [12]:
# Output reports

print("_______________________________________________________________________________________________")
print(f"The production of goods by industry and year for the dynamic Leontief model of the economy is:")
print("_______________________________________________________________________________________________")

goods = {}
 
for i in industries:
    my_list = []
    for t in fiveYears:
        my_list.append('${:,.2f}'.format( production[i,t ].x ) )
    goods[i] = my_list

goods_production = pd.DataFrame(goods, index=["Year1", "Year2", "Year3", "Year4", "Year5"])
goods_production

_______________________________________________________________________________________________
The production of goods by industry and year for the dynamic Leontief model of the economy is:
_______________________________________________________________________________________________


,coal,steel,transport
Year1,$0.00,$0.00,$0.00
Year2,$300.00,$136.78,$211.30
Year3,$370.87,$231.91,$217.39
Year4,$368.03,$209.57,$275.67
Year5,$961.06,$350.00,$92.31


In [13]:
print("_______________________________________________________________________________________________")
print(f"The productive capacity by industry and year for the dynamic Leontief model of the economy is:")
print("_______________________________________________________________________________________________")

# Compute cummulative capacity
totalCap = {}
 
for i in industries:
    amount = capacity0[i]
    my_list = []
    for t in fiveYears:
        amount += extra_cap[i,t].x
        my_list.append(round(amount))
    totalCap[i] = my_list
    
extra_capacity = pd.DataFrame(totalCap, index=["Year1", "Year2", "Year3", "Year4", "Year5"])
extra_capacity

_______________________________________________________________________________________________
The productive capacity by industry and year for the dynamic Leontief model of the economy is:
_______________________________________________________________________________________________


,coal,steel,transport
Year1,300,350,280
Year2,300,350,280
Year3,371,350,280
Year4,371,350,280
Year5,961,350,280


In [14]:
print("____________________________________________________________________________________")
print(f"Stock level by industry at the end of year for the dynamic Leontif model of the economy is:")
print("____________________________________________________________________________________")

inv = {}
 
for i in industries:
    my_list = []
    for t in fiveYears:
        my_list.append('${:,.2f}'.format( stock[i,t ].x ) )
    inv[i] = my_list

stock_level = pd.DataFrame(inv, index=["Year1", "Year2", "Year3", "Year4", "Year5"])
stock_level

____________________________________________________________________________________
Stock level by industry at the end of year for the dynamic Leontif model of the economy is:
____________________________________________________________________________________


,coal,steel,transport
Year1,$0.00,$26.97,$39.89
Year2,$0.00,$0.00,$80.34
Year3,$0.00,$0.00,$0.00
Year4,$0.00,$0.00,$0.00
Year5,$794.66,$244.33,$0.00


## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.

Wagner, H.M. (1957) A linear programming solution to dynamic Leontief type models. Management Science, 3, 234–254.

Copyright © 2020 Gurobi Optimization, LLC